In [237]:
import pandas as pd
import numpy as np
import ast

# train_full_class_df = pd.read_csv("/content/drive/MyDrive/train_new.csv")
train_df = pd.read_csv("/content/drive/MyDrive/test_new.csv")
val_df = pd.read_csv("/content/drive/MyDrive/val_pipeline.csv")

In [238]:
train_df = train_df.drop(columns=['image_url', 'video_thumbnail_url', 'video_url',
                    'gif_thumbnail_url', 'gif_url', 'is_url_active', 'date', 'media', 'Unnamed: 0', 'text_status', 'image_url_status', 'video_url_status', 'video_thumbnail_status','gif_thumbnail_status', 'gif_url_status', 'username', 'inferred company'])

train_df['video_bitrate'] = train_df['video_bitrate'].replace("Does not exist", 0)
train_df['video_bitrate'] = train_df['video_bitrate'].replace("0", 0)
train_df['video_duration'] = train_df['video_duration'].replace("Does not exist", 0)
train_df['video_duration'] = train_df['video_duration'].replace("0", 0)
train_df['video_views'] = train_df['video_views'].replace("Does not exist", 0)
train_df['video_views'] = train_df['video_views'].replace("0", 0)

train_df['gif_bitrate'] = train_df['gif_bitrate'].replace("Does not exist", 0)
train_df['gif_bitrate'] = train_df['gif_bitrate'].replace("0", 0)

train_df['video_bitrate'] = train_df['video_bitrate'].astype(float)
train_df['video_duration'] = train_df['video_duration'].astype(float)
train_df['video_views'] = train_df['video_views'].astype(float)
train_df['gif_bitrate'] = train_df['gif_bitrate'].astype(float)

def convert_to_tuple(string):
    return ast.literal_eval(string)

train_df['image_size'] = train_df['image_size'].fillna('(0, 0)')
train_df['image_size'] = train_df['image_size'].apply(convert_to_tuple)

train_df[['image_heigth', 'image_width']] = pd.DataFrame(train_df['image_size'].tolist(), columns=['image_height', 'image_heigth'])

train_df.drop('image_size', axis=1, inplace=True)
train_df['languages'] = train_df['languages'].str.strip("[]")
train_df['languages'] = train_df['languages'].str.strip("'")

for language in train_df['languages'].unique():
  if language != 'ENGLISH' and language != 'Unknown':
    train_df['languages'] = train_df['languages'].replace(language, "Another Language")

train_df = train_df.drop(columns=['content'])
likes_list = train_df['likes']

train_df = pd.get_dummies(train_df)
len(train_df.columns)

41

In [239]:
labels  = []
count_1 = 0
count_0 = 0
for index, row in train_df.iterrows():
    if likes_list[index] > 10000:
      labels.append(1)
      count_1 += 1
    else:
      labels.append(0)
      count_0 += 1
print(count_1, count_0)

677 53654


In [240]:
df_is_outlier = train_df[train_df['likes'] >= 10000]
df_is_not_outlier = train_df[train_df['likes'] < 10000]

In [241]:
len(df_is_outlier), len(df_is_not_outlier)

(677, 53654)

In [242]:
df_is_not_outlier = df_is_not_outlier.sample(len(df_is_outlier)*2)
len(df_is_not_outlier)

1354

In [243]:
df_is_outlier['outlier_label'] = 1
df_is_not_outlier['outlier_label'] = 0

<ipython-input-243-cae4b3bd5334>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_is_outlier['outlier_label'] = 1


In [244]:
df_is_outlier.head()

,video_bitrate,video_duration,video_views,gif_bitrate,year,day_of_year,likes,emoji_count,hastag_count,uppercase_count,...,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,media_type_gif,media_type_image,media_type_video,outlier_label
91,432000.0,65.300,142847.0,0.0,2020,42,11123,1,1,8,...,0,0,0,0,1,0,0,0,1,1
226,432000.0,137.650,116905.0,0.0,2020,271,11629,2,0,2,...,0,0,1,0,0,0,0,0,1,1
499,288000.0,85.953,363315.0,0.0,2020,64,12942,0,0,0,...,0,0,0,0,0,1,0,0,1,1
524,432000.0,14.982,432637.0,0.0,2020,279,16511,1,1,0,...,1,0,0,0,0,0,0,0,1,1
526,0.0,0.000,0.0,0.0,2020,305,19442,14,7,5,...,0,1,0,0,0,0,0,1,0,1


In [245]:
full_class_df = pd.concat([df_is_outlier, df_is_not_outlier], axis=0)
len(full_class_df)

2031

In [246]:
full_class_df = full_class_df.sample(len(full_class_df))
full_class_df.head()

,video_bitrate,video_duration,video_views,gif_bitrate,year,day_of_year,likes,emoji_count,hastag_count,uppercase_count,...,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,media_type_gif,media_type_image,media_type_video,outlier_label
15430,0.0,0.000,0.0,0.0,2020,9,54,1,1,0,...,0,0,0,1,0,0,0,1,0,0
43429,256000.0,60.825,319149.0,0.0,2018,300,13328,2,1,3,...,0,1,0,0,0,0,0,0,1,1
46897,0.0,0.000,0.0,0.0,2020,288,207,0,0,0,...,0,0,0,0,0,1,0,1,0,0
14061,320000.0,43.477,219396.0,0.0,2018,76,694,4,0,0,...,0,1,0,0,0,0,0,0,1,0
19499,256000.0,32.830,1056582.0,0.0,2018,138,23747,0,0,1,...,0,0,0,0,0,0,0,0,1,1


In [247]:
full_class_df = full_class_df.drop(columns=['likes'])

In [248]:
full_class_df.head()

,video_bitrate,video_duration,video_views,gif_bitrate,year,day_of_year,emoji_count,hastag_count,uppercase_count,at_count,...,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,media_type_gif,media_type_image,media_type_video,outlier_label
15430,0.0,0.000,0.0,0.0,2020,9,1,1,0,0,...,0,0,0,1,0,0,0,1,0,0
43429,256000.0,60.825,319149.0,0.0,2018,300,2,1,3,0,...,0,1,0,0,0,0,0,0,1,1
46897,0.0,0.000,0.0,0.0,2020,288,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
14061,320000.0,43.477,219396.0,0.0,2018,76,4,0,0,0,...,0,1,0,0,0,0,0,0,1,0
19499,256000.0,32.830,1056582.0,0.0,2018,138,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


In [249]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import QuantileTransformer

target = 'outlier_label'

full_class_df = full_class_df

X = full_class_df.drop(target, axis=1)
y = full_class_df[target]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_test, label=y_test)

params = {
    'max_depth': 3,
    'eta': 0.1,
    'objective': 'binary:logistic'
}

num_round = 100
model = xgb.train(params, dtrain, num_round)

predictions = model.predict(dtest)

predicted_classes = [1 if pred > 0.5 else 0 for pred in predictions]

accuracy = accuracy_score(y_test, predicted_classes)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8574938574938575


In [251]:
model.save_model('/content/drive/MyDrive/xg_boost_outlier_score_model.bin')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [16:25:43] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
